In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score
import math
import random
import time
from tkinter import *
import statistics 
from statistics import mode 

# fetching data and converting to list excluding labels
df=pd.read_csv('C:/Users/shrut/Documents/MachineLearning/HW6/iris.csv')
df.head()
labels = df['Species'].values.tolist()
labels
df.dropna()
data=df.drop(['Species'], axis=1)
data
list=data.values.tolist()
list

#Euclidean distance
def euclidean(centroid, data):
    sum1=[]
    if len(centroid) == len(data)+1:
        centroid=centroid[1:]
    for i in range (0, len(data)):
        sum1.append((centroid[i]- data[i])**2)
    euclidean= math.sqrt(sum(sum1))
    return euclidean

#Jaccard Distance
def jaccard(centroid, data):
    min_value=[]
    max_value = []
    if len(centroid) == 5:
        centroid = centroid[1:]
    for i in range(len(data)):
        min_value.append(min(centroid[i],data[i]))
        max_value.append(max(centroid[i],data[i]))
    return 1-(sum(min_value)/sum(max_value))

#Cosine distance
def cosine(centroid, data):
    if len(centroid) == 5:
        centroid = centroid[1:]
    x1 = centroid
    x1 = np.array(x1)
    x1 = x1.reshape(1,-1)
    y1 = np.array(data)
    y1 = y1.reshape(1,-1)
    result = cosine_similarity(x1,y1)
    return 1-result[0][0]

# Loading file

def loadCSV(fileName):
    fileHandler = open(fileName, "rt")
    lines = fileHandler.readlines()
    fileHandler.close()
    del lines[0] # remove the header
    dataset = []
    for line in lines:
        instance = lineToTuple(line)
        dataset.append(instance)
    return dataset

#Converting to numbers
def stringsToNumbers(myList):
    for i in range(len(myList)):
        if (isValidNumberString(myList[i])):
            myList[i] = float(myList[i])
            
#distance measurement           
def distance(instance1, instance2):
    if instance1 == None or instance2 == None:
        return float("inf")
    sumOfSquares = 0
    for i in range(0, len(instance1)-1):
        sumOfSquares += (instance1[i+1] - instance2[i])**2
    return sumOfSquares

#mean value
def meanInstance(name, instanceList):
    numInstances = len(instanceList)
    if (numInstances == 0):
        return
    numAttributes = len(instanceList[0])
    means = [name] + [0] * (numAttributes)
    for instance in instanceList:
        for i in range(0, numAttributes):
            means[i+1] += instance[i]
   
    for i in range(0, numAttributes):
        means[i+1] /= float(numInstances)
    return tuple(means)

#for the instance assigning centroids and distance measurement
def assign(instance, centroids, distance):
    minDistance = distance(centroids[0],instance)
    minDistanceIndex = 0
    for i in range(1, len(centroids)):
        d = distance(centroids[i], instance )
        if (d < minDistance):
            minDistance = d
            minDistanceIndex = i
    return minDistanceIndex

#creating empty list to store values
def createEmptyListOfLists(numSubLists):
    myList = []
    for i in range(numSubLists):
        myList.append([])
    return myList

#asiigning for all
def assignAll(instances, centroids, labels, distance):
    clusters = createEmptyListOfLists(len(centroids))
    classclusters = createEmptyListOfLists(len(centroids))
    i=0
    for instance in instances:
        clusterIndex = assign(instance, centroids, distance)
        clusters[clusterIndex].append(instance)
        classclusters[clusterIndex].append(labels[i])
        i=i+1
    return clusters, classclusters

#calculating centroids
def computeCentroids(clusters):
    centroids = []
    for i in range(len(clusters)):
        name = "centroid" + str(i)
        centroid = meanInstance(name, clusters[i])
        centroids.append(centroid)
    return centroids

# Defining kmeans function
def kmeans(instances, k, distance,labels, animation=False, initCentroids=None):
    result = {}
    if (initCentroids == None or len(initCentroids) < k):
        # randomly select k initial centroids
        random.seed(time.time())
        centroids = random.sample(instances, k)
    else:
        centroids = initCentroids
    prevCentroids = []
    if animation:
        delay = 1.0 # seconds
        clusters = createEmptyListOfLists(k)
        clusters[0] = instances
    iteration = 0
    while (centroids != prevCentroids):
        iteration += 1
        clusters, required_labels = assignAll(instances, centroids, labels, distance)
        prevCentroids = centroids
        centroids = computeCentroids(clusters)
        withinss = computeWithinss(clusters, centroids, distance)
    print('Number of iterations are ', iteration)
    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    result["labels"] = required_labels
    return result

#within the cluster distance
def computeWithinss(clusters, centroids, distance):
    result = 0
    for i in range(len(centroids)):
        centroid = centroids[i]
        cluster = clusters[i]
        for instance in cluster:
            result += distance(centroid, instance)
    return result

# Repeats k-means clustering n times, and returns the clustering
# with the smallest withinss
def repeatedKMeans(instances, k, n):
    bestClustering = {}
    bestClustering["withinss"] = float("inf")
    for i in range(1, n+1):
        print ("k-means trial %d," % i )
        trialClustering = kmeans(instances, k)
        print ("withinss: %.1f" % trialClustering["withinss"])
        if trialClustering["withinss"] < bestClustering["withinss"]:
            bestClustering = trialClustering
            minWithinssTrial = i
    print("Trial with minimum withinss:", minWithinssTrial)
    return bestClustering

#printing Table for clusters
def printTable(instances):
    for instance in instances:
        if instance != None:
            line = instance[0] + "\t"
            for i in range(1, len(instance)):
                line += "%.2f " % instance[i]
            print(line)

#extracting attributes
def extractAttribute(instances, index):
    result = []
    for instance in instances:
        result.append(instance[index])
    return result

#painting the clusters
def paintCircle(canvas, xc, yc, r, color):
    canvas.create_oval(xc-r, yc-r, xc+r, yc+r, outline=color)

def paintSquare(canvas, xc, yc, r, color):
    canvas.create_rectangle(xc-r, yc-r, xc+r, yc+r, fill=color)

def drawPoints(canvas, instances, color, shape):
    random.seed(0)
    width = canvas.winfo_reqwidth()
    height = canvas.winfo_reqheight()
    margin = canvas.data["margin"]
    minX = canvas.data["minX"]
    minY = canvas.data["minY"]
    maxX = canvas.data["maxX"]
    maxY = canvas.data["maxY"]
    scaleX = float(width - 2*margin) / (maxX - minX)
    scaleY = float(height - 2*margin) / (maxY - minY)
    for instance in instances:
        x = 5*(random.random()-0.5)+margin+(instance[1]-minX)*scaleX
        y = 5*(random.random()-0.5)+height-margin-(instance[2]-minY)*scaleY
        if (shape == "square"):
            paintSquare(canvas, x, y, 5, color)
        else:
            paintCircle(canvas, x, y, 5, color)
    canvas.update()

def connectPoints(canvas, instances1, instances2, color):
    width = canvas.winfo_reqwidth()
    height = canvas.winfo_reqheight()
    margin = canvas.data["margin"]
    minX = canvas.data["minX"]
    minY = canvas.data["minY"]
    maxX = canvas.data["maxX"]
    maxY = canvas.data["maxY"]
    scaleX = float(width - 2*margin) / (maxX - minX)
    scaleY = float(height - 2*margin) / (maxY - minY)
    for p1 in instances1:
        for p2 in instances2:
            x1 = margin + (p1[1]-minX)*scaleX
            y1 = height - margin - (p1[2]-minY)*scaleY
            x2 = margin + (p2[1]-minX)*scaleX
            y2 = height - margin - (p2[2]-minY)*scaleY
            canvas.create_line(x1, y1, x2, y2, fill=color)
    canvas.update()

def mergeClusters(clusters):
    result = []
    for cluster in clusters:
        result.extend(cluster)
    return result

def prepareWindow(instances):
    width = 500
    height = 500
    margin = 50
    root = Tk()
    canvas = Canvas(root, width=width, height=height, background="white")
    canvas.pack()
    canvas.data = {}
    canvas.data["margin"] = margin
    setBounds2D(canvas, instances)
    paintAxes(canvas)
    canvas.update()
    return canvas

def setBounds2D(canvas, instances):
    attributeX = extractAttribute(instances, 1)
    attributeY = extractAttribute(instances, 2)
    canvas.data["minX"] = min(attributeX)
    canvas.data["minY"] = min(attributeY)
    canvas.data["maxX"] = max(attributeX)
    canvas.data["maxY"] = max(attributeY)

def paintAxes(canvas):
    width = canvas.winfo_reqwidth()
    height = canvas.winfo_reqheight()
    margin = canvas.data["margin"]
    minX = canvas.data["minX"]
    minY = canvas.data["minY"]
    maxX = canvas.data["maxX"]
    maxY = canvas.data["maxY"]
    canvas.create_line(margin/2, height-margin/2, width-5, height-margin/2,
                       width=2, arrow=LAST)
    canvas.create_text(margin, height-margin/4,
                       text=str(minX), font="Sans 11")
    canvas.create_text(width-margin, height-margin/4,
                       text=str(maxX), font="Sans 11")
    canvas.create_line(margin/2, height-margin/2, margin/2, 5,
                       width=2, arrow=LAST)
    canvas.create_text(margin/4, height-margin,
                       text=str(minY), font="Sans 11", anchor=W)
    canvas.create_text(margin/4, margin,
                       text=str(maxY), font="Sans 11", anchor=W)
    canvas.update()


def showDataset2D(instances):
    canvas = prepareWindow(instances)
    paintDataset2D(canvas, instances)

def paintDataset2D(canvas, instances):
    canvas.delete(ALL)
    paintAxes(canvas)
    drawPoints(canvas, instances, "blue", "circle")
    canvas.update()

def showClusters2D(clusteringDictionary):
    clusters = clusteringDictionary["clusters"]
    centroids = clusteringDictionary["centroids"]
    withinss = clusteringDictionary["withinss"]
    canvas = prepareWindow(mergeClusters(clusters))
    paintClusters2D(canvas, clusters, centroids,
                    "Withinss: %.1f" % withinss)

def paintClusters2D(canvas, clusters, centroids, title=""):
    canvas.delete(ALL)
    paintAxes(canvas)
    colors = ["blue", "red", "green", "brown", "purple", "orange"]
    for clusterIndex in range(len(clusters)):
        color = colors[clusterIndex%len(colors)]
        instances = clusters[clusterIndex]
        centroid = centroids[clusterIndex]
        drawPoints(canvas, instances, color, "circle")
        if (centroid != None):
            drawPoints(canvas, [centroid], color, "square")
        connectPoints(canvas, [centroid], instances, color)
    width = canvas.winfo_reqwidth()
    canvas.create_text(width/2, 20, text=title, font="Sans 14")
    canvas.update()
    
#SSE calculation
def sse(clustering, centroids):
    i=0
    SSE=0
    for clust in clustering:
        for data in clust:
            SSE=SSE+distance(centroids[i], data)
        i=i+1
    return SSE

#calculating accuracy
def accuracy(clustering):
    correct_pred = 0
    for clust in clustering['labels']:
        for label in clust:
            if label == mode(clust):
                correct_pred+=1
    return correct_pred/150

#Testing and calculating SSE and accuracy using Cosine distance metric
clustering = kmeans(list, 3, cosine,labels, True)
printTable(clustering["centroids"])
print('Cosine_SSE is ',sse(clustering['clusters'],clustering['centroids']))
print("Cosine_Accuracy is ",accuracy(clustering))

#Testing and calculating SSE and accuracy using Jaccard distance metric
clustering = kmeans(list, 3, jaccard,labels, True)
printTable(clustering["centroids"])
print('Jaccard_SSE is ',sse(clustering['clusters'],clustering['centroids']))
print("Jaccard_Accuracy is ",accuracy(clustering))

#Testing and calculating SSE and accuracy using Euclidean distance metric
clustering = kmeans(list, 3, euclidean,labels, True)
printTable(clustering["centroids"])
print('Euclidean_SSE is ',sse(clustering['clusters'],clustering['centroids']))
print("Euclidean_Accuracy is ",accuracy(clustering))

# defining kmeans to calculate SSE for different distance metrics for different iterations
def kmeans_sse(instances, k, distance,labels, animation=False, initCentroids=None):
    result = {}
    if (initCentroids == None or len(initCentroids) < k):
        # randomly select k initial centroids
        random.seed(time.time())
        centroids = random.sample(instances, k)
    else:
        centroids = initCentroids
    prevCentroids = [] 
    new_sse= 99999 
    prev_sse=1000000 
    if animation:
        delay = 1.0 # seconds
        clusters = createEmptyListOfLists(k)
        clusters[0] = instances
    iteration = 0
    while (new_sse < prev_sse):
        iteration += 1
        clusters, required_labels = assignAll(instances, centroids, labels, distance)
        prev_sse = new_sse
        centroids = computeCentroids(clusters)
        new_sse= sse(clusters, centroids)
        withinss = computeWithinss(clusters, centroids, distance)
    print('Number of iterations are ', iteration)
    print('prev sse', prev_sse)
    print('new sse', new_sse)
    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    result["labels"] = required_labels
    return result

#cosine distance metric
before= datetime.now()
clustering = kmeans_sse(list, 3, cosine,labels, True)
after= datetime.now()
printTable(clustering["centroids"])
print("delay", after-before)
print('Cosine_SSE_iteration is ',sse(clustering['clusters'],clustering['centroids']))
print("Cosine_Iteration_Accuracy is ",accuracy(clustering))

#Jaccard distance metric
before= datetime.now()
clustering = kmeans_sse(list, 3, cosine,labels, True)
after= datetime.now()
printTable(clustering["centroids"])
print("delay is ", after- before)
print('Jaccard_iteration_SSE is ',sse(clustering['clusters'],clustering['centroids']))
print("Jaccard_Accuracy_iteration is ",accuracy(clustering))

#Euclidean distance metric
before= datetime.now()
clustering = kmeans_sse(list, 3, euclidean,labels, True)
after= datetime.now()
printTable(clustering["centroids"])
print("delay is ", after- before)
print('Euclidean_iteration_SSE is ',sse(clustering['clusters'],clustering['centroids']))
print("Euclidean_iteration_Accuracy is ",accuracy(clustering))

# defining kmeans to calculate SSE for different distance metrics for 100 iterations
def kmeans_100_iterations(instances, k, distance,labels, animation=False, initCentroids=None):
    result = {}
    if (initCentroids == None or len(initCentroids) < k):
        random.seed(time.time())
        centroids = random.sample(instances, k)
    else:
        centroids = initCentroids
    prevCentroids = []
    if animation:
        delay = 1.0 # seconds
        clusters = createEmptyListOfLists(k)
        clusters[0] = instances
    iteration = 0
    while (iteration<100):
        iteration +=1
        clusters, required_labels = assignAll(instances, centroids, labels, distance)
        prevCentroids = centroids
        centroids = computeCentroids(clusters)
        withinss = computeWithinss(clusters, centroids, distance)
    print('Number of iterations are ', iteration)
    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    result["labels"] = required_labels
    return result

#Cosine distance metric
before= datetime.now()
clustering = kmeans_100_iterations(list, 3, cosine,labels, True)
after= datetime.now()
printTable(clustering["centroids"])
print("delay is ", after- before)
print('Cosine_100_SSE is ',sse(clustering['clusters'],clustering['centroids']))
print("Cosine_100_Accuracy is ",accuracy(clustering))

#Jaccard distance metric
before= datetime.now()
clustering = kmeans_100_iterations(list, 3, jaccard,labels, True)
after= datetime.now()
printTable(clustering["centroids"])
print("delay is ", after- before)
print('Jaccard_100_SSE is ',sse(clustering['clusters'],clustering['centroids']))
print("Jaccard_Accuracy is ",accuracy(clustering))

#Euclidean distance metric
before= datetime.now()
clustering = kmeans_100_iterations(list, 3, euclidean,labels, True)
after= datetime.now()
printTable(clustering["centroids"])
print("delay is ", after- before)
print('Euclidean_100_SSE is ',sse(clustering['clusters'],clustering['centroids']))
print("Euclidean_100_Accuracy is ",accuracy(clustering))


Number of iterations are  5
centroid0	5.01 3.43 1.46 0.25 
centroid1	5.95 2.76 4.20 1.30 
centroid2	6.52 2.97 5.48 1.99 
Cosine_SSE is  93.77039393939394
Cosine_Accuracy is  0.9666666666666667
Number of iterations are  7
centroid0	5.89 2.74 4.41 1.43 
centroid1	6.87 3.09 5.72 2.07 
centroid2	5.01 3.43 1.46 0.25 
Jaccard_SSE is  79.0973497453311
Jaccard_Accuracy is  0.88
Number of iterations are  8
centroid0	5.88 2.74 4.39 1.43 
centroid1	6.85 3.08 5.72 2.05 
centroid2	5.01 3.43 1.46 0.25 
Euclidean_SSE is  78.85566582597733
Euclidean_Accuracy is  0.8866666666666667
Number of iterations are  6
prev sse 152.16353623188402
new sse 152.16353623188402
centroid0	5.10 3.65 1.38 0.24 
centroid1	4.93 3.24 1.53 0.25 
centroid2	6.26 2.87 4.91 1.68 
delay 0:00:00.762960
Cosine_SSE_iteration is  152.16353623188402


StatisticsError: no unique mode; found 2 equally common values